In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
pd.set_option('display.max_colwidth', 300)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import f1_score, accuracy_score

In [30]:
data = []

with open("C:/Users/Andrew/CSP571/Project/data/booksummaries.txt", 'r', encoding="utf-8") as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in reader:
        data.append(row)

#traits to record
bookID=[]
bookTitle=[]
bookSummary=[]
bookGenres=[]

In [33]:
for i in range(len(data)):
    try:
        #Json Handler
        bookGenres.append(list(json.loads(data[i][5]).values()))
    
        bookID.append(data[i][0])
        bookTitle.append(data[i][2])
        bookSummary.append(data[i][6])
    except:
        pass

bookDF=pd.DataFrame({'ID':bookID,'Title':bookTitle,'Genres':bookGenres,'Summary':bookSummary})

In [34]:
bookDF.head()

,ID,Title,Genres,Summary
0,620,Animal Farm,"[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p..."
1,843,A Clockwork Orange,"[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and..."
2,986,The Plague,"[Existentialism, Fiction, Absurdist fiction, Novel]","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t..."
3,2080,A Fire Upon the Deep,"[Hard science fiction, Science Fiction, Speculative fiction, Fantasy, Fiction]","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti..."
4,2152,All Quiet on the Western Front,"[War novel, Roman à clef]","The book tells the story of Paul Bäumer, a German soldier who—urged on by his school teacher—joins the German army shortly after the start of World War I. Bäumer arrives at the Western Front with his friends and schoolmates (Tjaden, Müller, Kropp and a number of other characters). There they me..."


In [35]:
#Get all unique genres (possibly use this to trim out certain genres?)
all_genres=sum(bookDF['Genres'],[])
len(set(all_genres))

In [44]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

#!pip install --upgrade gensim
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [47]:
#Text preprocessing
def preprocess(summ):
    stopWords=set(stopwords.words('english'))
    summ=re.sub("\'","",summ)
    summ=re.sub("[^a-zA-Z]"," ",summ)
    summ=' '.join([word for word in summ.split() if not word in stopWords])
    return summ.lower()

In [48]:
bookDF['Summary']=bookDF['Summary'].apply(lambda x: preprocess(x))

In [49]:
bookDF.head()

,ID,Title,Genres,Summary
0,620,Animal Farm,"[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]",old major old boar manor farm calls animals farm meeting compares humans parasites teaches animals revolutionary song beasts england major dies two young pigs snowball napoleon assume command turn dream philosophy animals revolt drive drunken irresponsible mr jones farm renaming animal farm adop...
1,843,A Clockwork Orange,"[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]",alex teenager living near future england leads gang nightly orgies opportunistic random ultra violence alexs friends droogs novels anglo russian slang nadsat dim slow witted bruiser gangs muscle georgie ambitious second command pete mostly plays along droogs indulge taste ultra violence characte...
2,986,The Plague,"[Existentialism, Fiction, Absurdist fiction, Novel]",text plague divided five parts town oran thousands rats initially unnoticed populace begin die streets hysteria develops soon afterward causing local newspapers report incident authorities responding public pressure order collection cremation rats unaware collection catalyst spread bubonic plagu...
3,2080,A Fire Upon the Deep,"[Hard science fiction, Science Fiction, Speculative fiction, Fantasy, Fiction]",novel posits space around milky way divided concentric layers called zones constrained different laws physics allowing different degrees biological technological advancement innermost unthinking depths surrounds galactic core incapable supporting advanced life forms next layer slow zone roughly ...
4,2152,All Quiet on the Western Front,"[War novel, Roman à clef]",book tells story paul b umer german soldier urged school teacher joins german army shortly start world war b umer arrives western front friends schoolmates tjaden ller kropp number characters meet stanislaus katczinsky older soldier nicknamed kat becomes pauls mentor fighting front b umer comrad...


In [50]:
#Multilabel Binarizer
labeler=MultiLabelBinarizer()
labeler.fit(bookDF['Genres'])
y=labeler.transform(bookDF['Genres'])

In [51]:
X_train,X_test,y_train,y_test=train_test_split(bookDF['Summary'],y,test_size=0.3)

In [52]:
tfidf=TfidfVectorizer(max_df=0.8,max_features=10000)
X_train=tfidf.fit_transform(X_train)
X_test=tfidf.fit_transform(X_test)

In [53]:
lr=LogisticRegression()
clf=OneVsRestClassifier(lr)
clf.fit(X_train,y_train)

C:\Users\Andrew\anaconda3\lib\site-packages\sklearn\multiclass.py:77: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
C:\Users\Andrew\anaconda3\lib\site-packages\sklearn\multiclass.py:77: UserWarning: Label not 15 is present in all training examples.
  str(classes[c]))
C:\Users\Andrew\anaconda3\lib\site-packages\sklearn\multiclass.py:77: UserWarning: Label not 22 is present in all training examples.
  str(classes[c]))
C:\Users\Andrew\anaconda3\lib\site-packages\sklearn\multiclass.py:77: UserWarning: Label not 59 is present in all training examples.
  str(classes[c]))
C:\Users\Andrew\anaconda3\lib\site-packages\sklearn\multiclass.py:77: UserWarning: Label not 66 is present in all training examples.
  str(classes[c]))
C:\Users\Andrew\anaconda3\lib\site-packages\sklearn\multiclass.py:77: UserWarning: Label not 84 is present in all training examples.
  str(classes[c]))
C:\Users\Andrew\anaconda3\lib\site-packages\sklearn\multiclass.py:77: UserWarning: Label 

OneVsRestClassifier(estimator=LogisticRegression())

In [55]:
y_hat=clf.predict(X_test)
accuracy_score(y_hat,y_test)

0.0018170019467878002